## 에이블 뉴스_ 오피니언 ver2
- https://www.ablenews.co.kr/
- 특성: 장애 전반의 고층과 사회 문제가 댓글/기사 많이 드러나서 데이터 품질도 높음.
- 타켓 :
    - 오피니언 > 세상이야기 > 정현석의 자취방 이야기
    - 오피니언 > 세상이야기 > 서인환의 회초리





- AbleNews SRN181 / SRN182 크롤러 + 2023년 이후 필터 + 워드클라우드

In [5]:
import time, os, traceback, re
from datetime import datetime

from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm

import pandas as pd
from konlpy.tag import Okt
import numpy as np

from wordcloud import WordCloud

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer



# ================================================================
# 1. 초기 설정
# ================================================================
CATEGORY_CODES = ["SRN182", "SRN181"]

MIN_YEAR = 2022
MIN_DATE = datetime(MIN_YEAR, 1, 1)

op_url = 'https://www.ablenews.co.kr/news/articleList.html?sc_sub_section_code=S2N3&view_type=sm'

BASE_LIST_URL = (
    "https://www.ablenews.co.kr/news/articleList.html"
    "?sc_serial_code={code}&sc_sub_section_code=S2N3&sc_section_code=S1N2"
    "&view_type=sm&page={page}"
)

MAX_PAGE = 15
KOREAN_FONT_PATH = "/System/Library/Fonts/AppleSDGothicNeo.ttc"

out_dir = "ablenews_result"
os.makedirs(out_dir, exist_ok=True)

error_log_file = os.path.join(out_dir, "ablenews_crawl_errors.log")
open(error_log_file, "w", encoding="utf-8").close()



# ================================================================
# 2. 에러 로그 함수
# ================================================================
def log_error(stage, msg="", cat=None, url=None, exc: Exception | None = None):
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    lines = [f"[{ts}] [{stage}]"]
    if cat:
        lines.append(f"category={cat}")
    if url:
        lines.append(f"url={url}")
    if msg:
        lines.append(f"msg={msg}")
    if exc:
        lines.append("traceback:")
        lines.append(traceback.format_exc())

    with open(error_log_file, "a", encoding="utf-8") as f:
        f.write(" | ".join(lines) + "\n")



# ================================================================
# 3. 날짜 파싱
# ================================================================
def parse_article_date(date_text: str):
    if not date_text:
        return None

    txt = date_text.strip()
    for prefix in ["입력", "등록", "수정"]:
        if txt.startswith(prefix):
            txt = txt[len(prefix):].strip()
            break

    txt = txt.replace("·", " ")
    txt = txt.replace("년", "-").replace("월", "-").replace("일", "")
    txt = txt.replace(".", "-").strip()

    for fmt in ["%Y-%m-%d %H:%M", "%Y-%m-%d"]:
        try:
            return datetime.strptime(txt, fmt)
        except:
            pass

    return None



# ================================================================
# 4. 크롬 드라이버 실행
# ================================================================
print("웹드라이버 실행 중...")
driver = wb.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get(op_url)
print("드라이버 준비 완료!\n")



# ================================================================
# 5. URL 수집 + 본문 수집
# ================================================================
all_frames = []

for cat_code in CATEGORY_CODES:
    print(f"\n===== 카테고리 [{cat_code}] 처리 시작 =====")
    page_url_list = []

    try:
        for page in tqdm(range(1, MAX_PAGE + 1), desc=f"[{cat_code}] 목록 페이지"):
            list_url = BASE_LIST_URL.format(code=cat_code, page=page)
            driver.get(list_url)
            time.sleep(1.5)

            article_links = driver.find_elements(By.CSS_SELECTOR,
                "#section-list ul.type2 li div.view-cont h4.titles a")

            new_cnt = 0
            for a in article_links:
                href = a.get_attribute("href")
                if href:
                    page_url_list.append(href)
                    new_cnt += 1

            if new_cnt == 0:
                print(f"⚠️ {page}페이지에서 새 URL 없음 → 중단")
                break

        page_url_list = list(dict.fromkeys(page_url_list))
        print(f"➡ 총 {len(page_url_list)}개 URL 수집됨")

    except Exception as e:
        log_error("URL 수집", "오류", cat=cat_code, exc=e)

    # 본문 크롤링 시작
    df_rows = []
    for href in tqdm(page_url_list, desc=f"[{cat_code}] 본문 수집"):
        try:
            driver.get(href)
            time.sleep(1.2)

            try:
                title = driver.find_element(By.CSS_SELECTOR,
                    "#articleViewCon > article.grid.body > header > h3").text.strip()
            except:
                title = None

            try:
                date_raw = driver.find_element(By.CSS_SELECTOR,
                    "#articleViewCon > article.grid.body > header > ul > li:nth-child(2)").text.strip()
            except:
                date_raw = None

            try:
                content = driver.find_element(By.CSS_SELECTOR,
                    "#article-view-content-div").text.strip()
            except:
                content = None

            parsed_date = parse_article_date(date_raw)

            if parsed_date and parsed_date < MIN_DATE:
                continue

            df_rows.append([cat_code, title, content, date_raw, parsed_date, href])

        except Exception as e:
            log_error("본문 크롤링", "오류", cat=cat_code, url=href, exc=e)

    col_names = ["category", "title", "content", "date_raw", "date_parsed", "url"]
    cat_df = pd.DataFrame(df_rows, columns=col_names)

    cat_df_clean = cat_df.dropna(subset=["title", "content", "date_raw"])
    cat_df_clean = cat_df_clean.drop_duplicates(subset=["url"])

    if not cat_df_clean.empty:
        # 카테고리별 CSV 저장
        per_cat_csv = os.path.join(out_dir, f"ablenews_{cat_code}.csv")
        cat_df_clean.to_csv(per_cat_csv, index=False, encoding="utf-8-sig")
        print(f"✅ [{cat_code}] 유효 데이터 {len(cat_df_clean)}건 → {per_cat_csv}")
        all_frames.append(cat_df_clean)
    else:
        log_error(stage="CSV 저장", msg="유효 데이터 없음 → CSV 미생성", cat=cat_code)
        print(f"⚠️ [{cat_code}] 유효 데이터가 없습니다.")


# ================================================================
# 6. 전체 데이터 합치기 + CSV, PKL 저장
# ================================================================
if all_frames:
    total_df = pd.concat(all_frames, ignore_index=True)
    total_df = total_df.drop_duplicates(subset=["url"]).copy()

    total_csv = os.path.join(out_dir, "ablenews_ALL.csv")
    total_pkl = os.path.join(out_dir, "ablenews_ALL.pkl")

    total_df.to_csv(total_csv, index=False, encoding="utf-8-sig")
    total_df.to_pickle(total_pkl)

    print(f"\n🎯 전체 합치기 완료 → {total_csv} (총 {len(total_df)}건)")
    print(f"💾 PKL 저장 완료 → {total_pkl}")

else:
    log_error(stage="최종 합치기", msg="합칠 데이터 없음")
    print("\n⚠️ 전체 합칠 데이터가 없습니다.")
    total_df = pd.DataFrame(columns=["category","title","content","date_raw","date_parsed","url"])




# ================================================================
# 7. 텍스트 전처리 / 품사 추출
# ================================================================
okt = Okt()

def clean_text(text: str):
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)
    text = re.sub(r'[^가-힣\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_korean_nouns(texts):
    nouns = []
    for t in texts:
        cleaned = clean_text(t)
        pos = okt.pos(cleaned, norm=True, stem=True)
        for word, tag in pos:
            if tag == "Noun":
                nouns.append(word)
    return " ".join(nouns)

def extract_korean_adjectives(texts):
    adjs = []
    for t in texts:
        cleaned = clean_text(t)
        pos = okt.pos(cleaned, norm=True, stem=True)
        for word, tag in pos:
            if tag == "Adjective":
                adjs.append(word)
    return " ".join(adjs)



# ================================================================
# 8. TF-IDF 기반 키워드 추출  ← 순서 먼저!
# ================================================================
def extract_keywords_tfidf(df, top_n=50):
    print("\n🔍 TF-IDF 키워드 추출 시작...")

    texts = df["content"].dropna().astype(str).tolist()
    corpus = [extract_korean_nouns([t]) for t in texts]

    vectorizer = TfidfVectorizer(max_features=5000, min_df=2, max_df=0.8)
    X = vectorizer.fit_transform(corpus)

    tfidf_scores = np.asarray(X.mean(axis=0)).ravel()
    vocab = vectorizer.get_feature_names_out()

    tfidf_df = pd.DataFrame({
        "word": vocab,
        "tfidf": tfidf_scores
    })

    top_keywords = tfidf_df.sort_values("tfidf", ascending=False).head(top_n)

    top_keywords.to_csv(os.path.join(out_dir, "ablenews_tfidf_keywords.csv"),
                        index=False, encoding="utf-8-sig")

    print("🎯 TF-IDF 키워드 추출 완료!")
    print(top_keywords)

    return top_keywords


tfidf_keywords = extract_keywords_tfidf(total_df, top_n=50)



# ================================================================
# 9. 워드클라우드 생성
# ================================================================
def create_wordclouds(df):
    texts = df["content"].dropna().astype(str).tolist()

    nouns_text = extract_korean_nouns(texts)
    adjs_text = extract_korean_adjectives(texts)

    wc_noun = WordCloud(
        font_path=KOREAN_FONT_PATH,
        width=1600, height=900,
        background_color="white"
    ).generate(nouns_text)
    wc_noun.to_file(os.path.join(out_dir, "ablenews_wordcloud_nouns.png"))

    wc_adj = WordCloud(
        font_path=KOREAN_FONT_PATH,
        width=1600, height=900,
        background_color="white"
    ).generate(adjs_text)
    wc_adj.to_file(os.path.join(out_dir, "ablenews_wordcloud_adjectives.png"))

    print("\n🎉 워드클라우드 생성 완료!")


create_wordclouds(total_df)


웹드라이버 실행 중...
드라이버 준비 완료!


===== 카테고리 [SRN182] 처리 시작 =====


[SRN182] 목록 페이지:  27%|█████▊                | 4/15 [00:26<01:12,  6.57s/it]


⚠️ 5페이지에서 새 URL 없음 → 중단
➡ 총 76개 URL 수집됨


[SRN182] 본문 수집: 100%|███████████████████████| 76/76 [02:57<00:00,  2.34s/it]


✅ [SRN182] 유효 데이터 41건 → ablenews_result/ablenews_SRN182.csv

===== 카테고리 [SRN181] 처리 시작 =====


[SRN181] 목록 페이지: 100%|█████████████████████| 15/15 [00:33<00:00,  2.26s/it]


➡ 총 300개 URL 수집됨


[SRN181] 본문 수집: 100%|█████████████████████| 300/300 [12:32<00:00,  2.51s/it]


✅ [SRN181] 유효 데이터 289건 → ablenews_result/ablenews_SRN181.csv

🎯 전체 합치기 완료 → ablenews_result/ablenews_ALL.csv (총 330건)
💾 PKL 저장 완료 → ablenews_result/ablenews_ALL.pkl

🔍 TF-IDF 키워드 추출 시작...
🎯 TF-IDF 키워드 추출 완료!
       word     tfidf
2093    서비스  0.041253
3513     장애  0.039696
4007     지원  0.032050
1923     사람  0.029423
2405  시각장애인  0.028455
471      교육  0.026911
3246    이야기  0.026578
4921     활동  0.026339
1974     사회  0.025836
2433     시설  0.024825
251      경우  0.024813
676      기술  0.024341
1949     사업  0.022518
3472     작가  0.022143
3250     이용  0.021555
1952     사용  0.021429
1447     문제  0.021353
2406     시간  0.020682
3306     인식  0.020066
135      개발  0.019484
1759     복지  0.019276
1007     대한  0.018843
2063     생각  0.018124
3445     자신  0.017990
42       가족  0.017966
3662    접근성  0.017888
1625     버스  0.017792
903      단체  0.017423
1168    디지털  0.017232
3684     정보  0.017190
1554   발달장애  0.017059
646      기관  0.016908
569      권리  0.016447
4704     학교  0.016086
372      공연  0.016074
